# APIs + OAI-PMH



### Librerías necesarias para la práctica

```python
import xml.etree.ElementTree as ET
import requests
import json
```

In [ ]:
import xml.etree.ElementTree as ET
import requests
import json

## Parte 1 - APIs - Swagger
Vamos a utilizar el ejemplo de la tienda de mascotas en swagger:
https://petstore.swagger.io

#### Ejercicio 1 - Lista 10 mascotas registradas

#### Ejercicio 2 - Registra una nueva mascota mascota

#### Ejercico 3 - Encuentra la mascota registrada con su ID

## Parte 2 - OAI-PMH
La documentación del protocolo la podéis encontrar aquí:

https://www.openarchives.org/OAI/openarchivesprotocol.html

### Identify

This verb is used to retrieve information about a repository. Some of the information returned is required as part of the OAI-PMH. Repositories may also employ the Identify verb to return additional descriptive information.

In [ ]:
import xml.etree.ElementTree as ET
import requests

base_url = "https://zenodo.org/oai2d"
action = "?verb=Identify"
print("Request to: %s%s" % (base_url, action))
oai = requests.get(base_url + action) #Peticion al servidor

#Para crear el arbol XML
xmlTree = ET.fromstring(oai.text)
print(oai.text)
#Iterator para recorrer todos los elementos (incluso anidados)
iterator = xmlTree.iter()
for elem in iterator:
    print(elem.tag+": "+elem.text)

#Findall para encontrar un elemento específico
namespaces = {'ns': 'http://www.openarchives.org/OAI/2.0/'} # add more as needed
elementos = xmlTree.findall('.//ns:baseURL',namespaces)
print(elementos) #Element
for e in elementos:
    print("BaseURL:", e.text)

### Listar tipos de metadatos

ListMetadataFormats

This verb is used to retrieve the metadata formats available from a repository. An optional argument restricts the request to the formats available for a specific item.


In [ ]:
import xml.etree.ElementTree as ET
import requests

base_url = 'https://zenodo.org/oai2d'
action = '?verb=ListMetadataFormats'

oai = requests.get(base_url + action) #Peticion al servidor

#Para crear el arbol XML
xmlTree = ET.fromstring(oai.text)

iterator = xmlTree.iter()
for elem in iterator:
    print(elem.tag+": "+elem.text)

Podemos capturar sólo la información relevante

In [ ]:
iterator = xmlTree.iter()
for elem in iterator:
    if (elem.tag == '{http://www.openarchives.org/OAI/2.0/}metadataPrefix') or (elem.tag == '{http://www.openarchives.org/OAI/2.0/}schema'):
        print(elem.tag+": "+elem.text)

O de forma más "elegante"

In [ ]:
for e in xmlTree.findall('.//{http://www.openarchives.org/OAI/2.0/}metadataFormat'):
    metadataPrefix = e.find('{http://www.openarchives.org/OAI/2.0/}metadataPrefix').text
    schema = e.find('{http://www.openarchives.org/OAI/2.0/}schema').text
    print(metadataPrefix, ':', schema)

Podemos describir el último schema

In [ ]:
oai_datacite = requests.get('http://schema.datacite.org/meta/kernel-4.1/metadata.xsd') #Peticion al servidor
print(oai_datacite.text)

#Para crear el arbol XML
xmlTree = ET.fromstring(oai_datacite.text)

iterator = xmlTree.iter()
for elem in iterator:
    print(elem.tag)

Si analizamos la descripción del estándar, vemos que las etiquetas de los metadatos están almacenados en la etiqueta "element", en el atributo "name". Podemos obetener un listado de lo que esperamos encontrar

In [ ]:
namespaces = {'xs' : 'http://www.w3.org/2001/XMLSchema'}
for tags in xmlTree.findall('.//xs:element',namespaces):
    print ('Metadato: ',tags.attrib['name'])

### ListIdentifiers

This verb is an abbreviated form of ListRecords, retrieving only headers rather than records. Optional arguments permit selective harvesting of headers based on set membership and/or datestamp. Depending on the repository's support for deletions, a returned header may have a status attribute of "deleted" if a record matching the arguments specified in the request has been deleted.

Devuelve la cabecera de los registros, para conocer la información esencial.

Necesita el prefijo del formato de metadatos

In [ ]:
#prefijo: oai_datacite4
base_url = 'https://zenodo.org/oai2d'
action = '?verb=ListIdentifiers'
params =  '&metadataPrefix=oai_datacite4'

oai = requests.get(base_url + action + params)
xmlTree = ET.fromstring(oai.text)

iterator = xmlTree.iter()
for elem in iterator:
    print(elem.tag+": "+elem.text) #Identifiers are really important because they are used to ask more information

### ListRecords

Listar los registros

This verb is used to harvest records from a repository. Optional arguments permit selective harvesting of records based on set membership and/or datestamp. Depending on the repository's support for deletions, a returned header may have a status attribute of "deleted" if a record matching the arguments specified in the request has been deleted. No metadata will be present for records with deleted status.

In [ ]:
import xml.etree.ElementTree as ET
import requests

base_url = 'https://zenodo.org/oai2d'
action = '?verb=ListRecords'
params =  '&metadataPrefix=oai_datacite4'

oai = requests.get(base_url + action + params)

xmlTree = ET.fromstring(oai.text)
iterator = xmlTree.iter()
for elem in iterator:
    print(elem.tag)
    print(elem.text)

## Ejercicio 1

Muestra sólo el identificador y las palabras clave (subject)

## Filtros

Registros entre dos fechas

In [ ]:
import xml.etree.ElementTree as ET

base_url = 'https://zenodo.org/oai2d'
action = '?verb=ListRecords'
params =  '&metadataPrefix=oai_datacite4'
bounds = "&from='2019-01-01'&until='2019-12-31'" #Para filtrar por fechas

oai = requests.get(base_url + action + params + bounds)
xmlTree = ET.fromstring(oai.text)
iterator = xmlTree.iter()
for elem in iterator:
    print(elem.tag)
    print(elem.text)

### GetRecord

This verb is used to retrieve an individual metadata record from a repository. Required arguments specify the identifier of the item from which the record is requested and the format of the metadata that should be included in the record. Depending on the level at which a repository tracks deletions, a header with a "deleted" value for the status attribute may be returned, in case the metadata format specified by the metadataPrefix is no longer available from the repository or from the specified item.

Obtener objeto digital de ese recurso

In [ ]:
import xml.etree.ElementTree as ET
import urllib

base_url = 'https://zenodo.org/oai2d'
action = '?verb=GetRecord'
params =  '&metadataPrefix=oai_datacite4&identifier=oai:zenodo.org:3363479'
print(base_url + action + params)
oai = requests.get(base_url + action + params)
print(oai.text)
xmlTree = ET.fromstring(oai.text)
ns = {'oai' : 'http://www.openarchives.org/OAI/2.0/'}

#iterator = xmlTree.iter()
#for elem in iterator:
 #   print(elem.tag)
 #   print(elem.text)

for tags in xmlTree.findall('.//',ns):
    print (tags.tag+": ",tags.text)

## APIs específicas

El protocolo OAI-PMH ofrece una serie de funcionalidades genéricas e interoperables entre distintos repositorios. Sin embargo, existen ciertas limitaciones que no nos permiten automatizar todo lo necesario para analizar un gran número de dataset. Para ello, tenemos que recurrir a APIs específicas de los repositorios con los que trabajamos.

Zenodo proporciona una API bastante completa que complementa al protocolo OAI-PMH. En el siguiente enlace podemos ver su especificación y algunos ejemplos con curl y python.

http://developers.zenodo.org/

Entre otras cosas, podemos automatizar el proceso de publicación, descargar archivos, etc.

También, obteniendo el DOI de un recurso, podemos resolverlo para obtener cierta información específica, como la URL del objeto digital entre otras.

In [ ]:
import json
#Buscamos datasets específicos-
oai = requests.get('https://zenodo.org/oai2d?verb=ListRecords&metadataPrefix=datacite3')
oaiTree = ET.fromstring(oai.text)
#for i in oaiTree.iter():
#    print(i.tag)
records = oaiTree.findall('.//{http://www.openarchives.org/OAI/2.0/}record')
for e in records:
    for subject in e.findall('.//{http://datacite.org/schema/kernel-3}subject'):
        if "Animalia" in subject.text:
            print("Title: "+e.findall('.//{http://datacite.org/schema/kernel-3}title')[0].text)
            identifier = e.findall('.//{http://datacite.org/schema/kernel-3}identifier')[0].text
            print(identifier)
            print('\n')
            
headers = {'accept': 'application/json'}
r = requests.get('https://zenodo.org/api/records/958140',headers)
record = json.loads(r.text)
print(record['files'])

    

Puedes mostrar directamente la imagen

In [ ]:
import requests
from PIL import Image
import matplotlib.pyplot as plt


# URL de la imagen
url = record['files'][0]['links']['self']

# Realizar la petición GET a la URL
respuesta = requests.get(url)

# Verificar que la petición fue exitosa
if respuesta.status_code == 200:
    with open('imagen_test.jpg', 'wb') as f:
        f.write(respuesta.content)
img = Image.open('imagen_test.jpg')
plt.imshow(img)
plt.axis('off')  # No mostrar ejes
plt.show()

# Ejercicio

* 1: Busca al menos dos repositorios digitales compatibles con OAI-PMH
* 2: Utiliza los verbos del protocolo para encontrar los recursos de un tema en concreto (puedes filtrar por la descripción o en las palabras clave).
* 3: Obtén un listado de los títulos, identificadores (DOI o cualquier otro) y la dirección del recurso.
* 4: ¿Qué problemas has encontrado? ¿Has tenido que manejar más de un formato de metadatos? ¿Qué limitaciones tiene el protocolo OAI-PMH? ¿Cómo lo mejorarías?
* 5: A través de los distintos métodos vistos (Uso de PID/DOI, OAI-PMH o APIs concretas de repositorios) intenta llegar al fichero de un dataset para mostrarlo (gráfica, imagen...). 